In [ ]:
import numpy as np
import pandas as pd
import os

# Открытие и загрузка данных
if not os.path.exists('weather-forecast-hackathon.zip'):
    !wget https://drive.google.com/uc?id=1Vun7-8vmYbSr4COqb1nAGogEIAzInxab -O weather-forecast-hackathon.zip
    !unzip weather-forecast-hackathon.zip
    !ls -l

data = {}
data_files = ['temperature.npy', 'pressure.npy', 'humidity.npy', 'wind_speed.npy', 'wind_dir.npy', 'cloud_cover.npy']
for file_name in data_files:
    data[file_name[:-4]] = np.load(file_name)

# Составление матрицы признаков X
X = np.stack([data[key] for key in data], axis=-1)
X = X.reshape(-1, 6)

# Разделение данных на X и y
y = X[34200:, :]
X = X[:-34200, :]
print("X shape:", X.shape)
print("y shape:", y.shape)

# Предобработка данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Добавление новых признаков (например, среднее значение и стандартное отклонение для каждого показателя)
X_additional = np.hstack((np.mean(X_scaled, axis=1).reshape(-1, 1), np.std(X_scaled, axis=1).reshape(-1, 1)))
X_scaled = np.hstack((X_scaled, X_additional))

# Разделение данных на обучающий и тестовый наборы
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Настройка гиперпараметров модели (XGBoost)
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.1, 0.3]
}

model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_percentage_error')
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

# Обучение модели с лучшими параметрами
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Оценка модели на тестовом наборе
from sklearn.metrics import mean_absolute_percentage_error
y_pred = best_model.predict(X_test)
mape_test = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE on test set:", mape_test)

# Предсказание для всех данных
predictions = best_model.predict(X_scaled)
print("Predictions shape:", predictions.shape)

# Сохранение предсказаний в CSV
solution = pd.DataFrame(predictions, columns=["temperature", "pressure", "humidity", "wind_speed", "wind_dir", "cloud_cover"])
solution.to_csv("solution.csv", index_label="ID")

X shape: (4500, 6)
y shape: (4500, 6)
Best parameters: {'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 300}
MAPE on test set: 79324780000.0
Predictions shape: (4500, 6)


In [ ]:
# import numpy as np
# import pandas as pd
# import os

# # Открытие и загрузка данных
# if not os.path.exists('weather-forecast-hackathon.zip'):
#     !wget https://drive.google.com/uc?id=1Vun7-8vmYbSr4COqb1nAGogEIAzInxab -O weather-forecast-hackathon.zip
#     !unzip weather-forecast-hackathon.zip
#     !ls -l

# data = {}
# data_files = ['temperature.npy', 'pressure.npy', 'humidity.npy', 'wind_speed.npy', 'wind_dir.npy', 'cloud_cover.npy']
# for file_name in data_files:
#     data[file_name[:-4]] = np.load(file_name)

# # Составление матрицы признаков X
# X = np.stack([data[key] for key in data], axis=-1)
# X = X.reshape(-1, 6)

# # Разделение данных на X и y
# y = X[34200:, :]
# X = X[:-34200, :]

# # Предобработка данных
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Добавление новых признаков (например, среднее значение и стандартное отклонение для каждого показателя)
# X_additional = np.hstack((np.mean(X_scaled, axis=1).reshape(-1, 1), np.std(X_scaled, axis=1).reshape(-1, 1)))
# X_scaled = np.hstack((X_scaled, X_additional))

# # Разделение данных на обучающий и тестовый наборы
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# # Настройка гиперпараметров модели (XGBoost) с помощью случайного поиска
# import xgboost as xgb
# from sklearn.model_selection import RandomizedSearchCV

# param_grid = {
#     'n_estimators': np.arange(100, 1000, 100),
#     'max_depth': np.arange(3, 10),
#     'learning_rate': np.linspace(0.01, 0.3, 10)
# }

# model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=50, cv=5, scoring='neg_mean_absolute_percentage_error', random_state=42)
# random_search.fit(X_train, y_train)

# print("Best parameters:", random_search.best_params_)

# # Обучение модели с лучшими параметрами
# best_model = random_search.best_estimator_
# best_model.fit(X_train, y_train)

# # Оценка модели на тестовом наборе
# from sklearn.metrics import mean_absolute_percentage_error
# y_pred = best_model.predict(X_test)
# mape_test = mean_absolute_percentage_error(y_test, y_pred)
# print("MAPE on test set:", mape_test)

# # Предсказание для всех данных
# predictions = best_model.predict(X_scaled)
# print("Predictions shape:", predictions.shape)

# # Сохранение предсказаний в CSV
# solution = pd.DataFrame(predictions, columns=["temperature", "pressure", "humidity", "wind_speed", "wind_dir", "cloud_cover"])
# solution.to_csv("solution.csv", index_label="ID")


Best parameters: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.23555555555555557}
MAPE on test set: 702859600000.0
Predictions shape: (4500, 6)
